# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [298]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [366]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [401]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位知识渊博的生物教师。
你擅长以清晰、简洁的方式回答关于生物学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_science_template = """你是一位优秀的计算机科学家。
你擅长解答计算机科学和编程相关的问题。
你的解答总是逻辑清晰，易于理解。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位卓有成效的汉语言文学教师。
你擅长解答关于汉语言文学的问题，无论是古代文学还是现代文学。
你的解答总是富有洞见，引人深思。

这是一个问题：
{input}"""


In [402]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机科学",
        "description": "适用于回答计算机科学和编程相关的问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答关于汉语言文学的问题",
        "prompt_template": chinese_literature_template,
    },
]

In [403]:
import os

api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",

             )

In [404]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [405]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [406]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [407]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [408]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机科学: 适用于回答计算机科学和编程相关的问题', '汉语言文学: 适用于回答关于汉语言文学的问题']


In [409]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学和编程相关的问题
汉语言文学: 适用于回答关于汉语言文学的问题


In [410]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [411]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学和编程相关的问题
汉语言文学: 适用于回答关于汉语言文学的问题

<< INPUT

In [412]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [413]:
print(chain.invoke("黑体辐射是什么？?"))




> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一种理想的物体，它可以完全吸收所有入射的辐射能量，并以各种波长的光线以及其他形式的辐射能量重新辐射出去。它是热力学中的一个概念，用于描述物体在各种温度下发出的辐射能量的分布。'}


In [414]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们要确定大于40的质数。质数是只能被1和自身整除的数，\n所以我们可以从41开始逐个判断是否为质数。\n41不被任何小于它的数整除，所以它是质数。\n接下来，我们需要找到一个质数加一后能被3整除的数。\n我们可以列举一些质数和它们加一后的数：2+1=3, 5+1=6, 7+1=8, 11+1=12。\n我们可以发现，只有6和12能被3整除。所以答案是41+1=42。\n42是大于40的第一个质数，且42+1=43能被3整除。'}


In [415]:
router_chain.verbose = True

In [416]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种天体，它具有非常强大的引力，甚至连光也无法逃脱。它是由大量物质被挤压在极小的空间内形成的，因此具有非常高的密度。黑洞的边界被称为事件视界，它是一种虚拟的表面，从这个表面向内部的任何物质都无法逃脱。目前科学家们还在研究黑洞的性质和行为，但它们在宇宙中扮演着重要的角色，影响着星系和宇宙的演化。'}


In [417]:
print(chain.invoke("DNA是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': 'DNA是什么？'}
> Finished chain.
{'input': 'DNA是什么？', 'text': '\n\nDNA是脱氧核糖核酸的缩写，是一种存在于细胞核和线粒体中的生物分子。它是生物体遗传信息的载体，通过它，生物体的遗传特征可以传递给后代。DNA由四种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和鳞腺嘧啶）构成的长链状分子，在细胞的复制和分裂过程中起着重要的作用。'}


In [420]:
print(chain.invoke("快速排序算法Java实现？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机科学: {'input': '快速排序算法Java实现？'}
> Finished chain.
{'input': '快速排序算法Java实现？', 'text': '\n\n快速排序是一种常用的排序算法，它的实现原理是利用分治的思想，将一个大的问题分解成多个小的子问题来解决。下面是使用Java语言实现快速排序的示例代码：\n\n// 快速排序函数，输入参数为待排序数组、起始位置和结束位置\npublic static void quickSort(int[] arr, int start, int end){\n    // 如果起始位置大于或等于结束位置，则说明已经排序完成\n    if (start >= end) {\n        return;\n    }\n    // 以数组中间的元素作为基准值\n    int pivot = arr[(start + end) / 2];\n    // 定义左指针和右指针\n    int left = start;\n    int right = end;\n    // 循环将数组中小于基准值的元素放在左边，大于基准值的元素放在右边\n    while (left <= right) {\n        // 在左边找到第一个大于基'}


In [419]:
print(chain.invoke("出师表起源"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '出师表起源'}
> Finished chain.
{'input': '出师表起源', 'text': '于哪个历史时期？\n\n出师表起源于三国时期。它是一篇由诸葛亮所作的文学作品，用于向刘备说明出兵北伐的重要性和必要性。这篇文学作品同时也展现了诸葛亮的治国思想和智慧，被后人称为“奇文”。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains